# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,36.68,88,2,11.88,GS,1696209778
1,1,bilibino,68.0546,166.4372,29.95,79,100,4.47,RU,1696209778
2,2,wailua homesteads,22.0669,-159.3780,86.43,83,40,11.50,US,1696209682
3,3,la sarre,48.8002,-79.1996,57.20,69,90,3.65,CA,1696209778
4,4,st. john's,47.5649,-52.7093,55.98,87,40,23.02,CA,1696209725


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Create a scatter plot using hvplot
city_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    cmap="tab20",
    geo = True,
    grid=True,
    tiles = "OSM",
    size=city_data_df["Humidity"] * 0.25,
    title="City Locations and Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
)

# Display the map plot_1
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Define your criteria for ideal weather conditions in Fahrenheit
max_temp_condition_F = (city_data_df["Max Temp"] >= 60) & (city_data_df["Max Temp"] <= 80)
humidity_condition = (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <= 60) 
wind_speed_condition = city_data_df["Wind Speed"] <= 10
cloudiness_condition = city_data_df["Cloudiness"] <= 50


# Apply the conditions to filter the DataFrame
ideal_weather_df_F = city_data_df[max_temp_condition_F & humidity_condition & wind_speed_condition & cloudiness_condition].copy()

# Drop any rows with null values if necessary
ideal_weather_df_F.dropna(inplace=True)

# Display sample data
ideal_weather_df_F.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,metkovic,43.0542,17.6483,61.27,52,0,6.38,HR,1696209779
58,58,nihtaur,29.3333,78.3833,71.85,58,30,6.38,IN,1696209784
153,153,saint-pierre,-21.3393,55.4781,71.28,60,0,9.22,RE,1696209793
168,168,chamical,-30.3600,-66.3140,67.84,41,0,9.91,AR,1696209674
254,254,silverton,45.0051,-122.7832,69.64,49,31,5.01,US,1696209803


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df_F[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,metkovic,HR,43.0542,17.6483,52,
58,nihtaur,IN,29.3333,78.3833,58,
153,saint-pierre,RE,-21.3393,55.4781,60,
168,chamical,AR,-30.3600,-66.3140,41,
254,silverton,US,45.0051,-122.7832,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius=10000
params = {
    "categories": "accommodation.hotel", # Search for lodging places
    "filter": "circle: radius",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Print a message to follow up the hotel search
    print("End of hotel search")
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

#AskBCS Learning Assistant helped to clarify and tweak the params

Starting hotel search
End of hotel search
metkovic - nearest hotel: Hotel MB
End of hotel search
nihtaur - nearest hotel: No hotel found
End of hotel search
saint-pierre - nearest hotel: Tropic Hotel
End of hotel search
chamical - nearest hotel: No hotel found
End of hotel search
silverton - nearest hotel: Silverton Inn & Suites
End of hotel search
hami - nearest hotel: Хами
End of hotel search
porto santo stefano - nearest hotel: Belvedere
End of hotel search
keren - nearest hotel: Eritrea Hotel قندق إرتريا
End of hotel search
portland - nearest hotel: the Hoxton
End of hotel search
sparti - nearest hotel: Maniatis
End of hotel search
connell - nearest hotel: No hotel found
End of hotel search
pahrump - nearest hotel: Holiday Inn Express & Suites
End of hotel search
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
End of hotel search
monteagudo - nearest hotel: Residencial San José
End of hotel search
montoro - nearest hotel: Hostal Restaurante Montoro
End of hotel search
laguna

,City,Country,Lat,Lng,Humidity,Hotel Name
6,metkovic,HR,43.0542,17.6483,52,Hotel MB
58,nihtaur,IN,29.3333,78.3833,58,No hotel found
153,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
168,chamical,AR,-30.3600,-66.3140,41,No hotel found
254,silverton,US,45.0051,-122.7832,49,Silverton Inn & Suites


In [7]:
# Filter the DataFrame to remove rows where "Hotel Name" is "No hotel found"
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Reset the index of the DataFrame
hotel_df.reset_index(drop=True, inplace=True)

# Display the updated DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,metkovic,HR,43.0542,17.6483,52,Hotel MB
1,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
2,silverton,US,45.0051,-122.7832,49,Silverton Inn & Suites
3,hami,CN,42.8000,93.4500,42,Хами
4,porto santo stefano,IT,42.4338,11.1237,56,Belvedere
5,keren,ER,15.7778,38.4581,38,Eritrea Hotel قندق إرتريا
6,portland,US,45.5234,-122.6762,51,the Hoxton
7,sparti,GR,37.0733,22.4297,60,Maniatis
8,pahrump,US,36.2083,-115.9839,32,Holiday Inn Express & Suites
9,kuqa,CN,41.7278,82.9364,39,Kuqa Hotel - foreigners accepted


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot for the hotels
hotel_map = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap="tab20",
    geo = True,
    tiles = "OSM",
    hover_cols=['City', 'Country', 'Hotel Name'],
    title='Hotel Locations',
    xlabel="Longitude",
    ylabel="Latitude",
    frame_width = 800,
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)